In [1]:
from transformers import pipeline

summarizer = pipeline("summarization", model="facebook/bart-large-xsum")

ARTICLE = """ New York (CNN)When Liana Barrientos was 23 years old, she got married in Westchester County, New York.
A year later, she got married again in Westchester County, but to a different man and without divorcing her first husband.
Only 18 days after that marriage, she got hitched yet again. Then, Barrientos declared "I do" five more times, sometimes only within two weeks of each other.
In 2010, she married once more, this time in the Bronx. In an application for a marriage license, she stated it was her "first and only" marriage.
Barrientos, now 39, is facing two criminal counts of "offering a false instrument for filing in the first degree," referring to her false statements on the
2010 marriage license application, according to court documents.
Prosecutors said the marriages were part of an immigration scam.
On Friday, she pleaded not guilty at State Supreme Court in the Bronx, according to her attorney, Christopher Wright, who declined to comment further.
After leaving court, Barrientos was arrested and charged with theft of service and criminal trespass for allegedly sneaking into the New York subway through an emergency exit, said Detective
Annette Markowski, a police spokeswoman. In total, Barrientos has been married 10 times, with nine of her marriages occurring between 1999 and 2002.
All occurred either in Westchester County, Long Island, New Jersey or the Bronx. She is believed to still be married to four men, and at one time, she was married to eight men at once, prosecutors say.
Prosecutors said the immigration scam involved some of her husbands, who filed for permanent residence status shortly after the marriages.
Any divorces happened only after such filings were approved. It was unclear whether any of the men will be prosecuted.
The case was referred to the Bronx District Attorney\'s Office by Immigration and Customs Enforcement and the Department of Homeland Security\'s
Investigation Division. Seven of the men are from so-called "red-flagged" countries, including Egypt, Turkey, Georgia, Pakistan and Mali.
Her eighth husband, Rashid Rajput, was deported in 2006 to his native Pakistan after an investigation by the Joint Terrorism Task Force.
If convicted, Barrientos faces up to four years in prison.  Her next court appearance is scheduled for May 18.
"""
print(summarizer(ARTICLE, max_length=130, min_length=30, do_sample=False))

[{'summary_text': 'A New York woman has pleaded not guilty to falsely claiming to be married 10 times, including to eight men from different countries, in what prosecutors say was an immigration scam.'}]


In [2]:
from datasets import load_dataset

dataset = load_dataset("xsum")

Found cached dataset xsum (/dgxa_home/se20ucse203/.cache/huggingface/datasets/xsum/default/1.2.0/082863bf4754ee058a5b6f6525d0cb2b18eadb62c7b370b095d1364050a52b71)


  0%|          | 0/3 [00:00<?, ?it/s]

In [3]:
dataset

DatasetDict({
    train: Dataset({
        features: ['document', 'summary', 'id'],
        num_rows: 204045
    })
    validation: Dataset({
        features: ['document', 'summary', 'id'],
        num_rows: 11332
    })
    test: Dataset({
        features: ['document', 'summary', 'id'],
        num_rows: 11334
    })
})

In [4]:
text = dataset['train']['document'][0]
print(text)

The full cost of damage in Newton Stewart, one of the areas worst affected, is still being assessed.
Repair work is ongoing in Hawick and many roads in Peeblesshire remain badly affected by standing water.
Trains on the west coast mainline face disruption due to damage at the Lamington Viaduct.
Many businesses and householders were affected by flooding in Newton Stewart after the River Cree overflowed into the town.
First Minister Nicola Sturgeon visited the area to inspect the damage.
The waters breached a retaining wall, flooding many commercial properties on Victoria Street - the main shopping thoroughfare.
Jeanette Tate, who owns the Cinnamon Cafe which was badly affected, said she could not fault the multi-agency response once the flood hit.
However, she said more preventative work could have been carried out to ensure the retaining wall did not fail.
"It is difficult but I do think there is so much publicity for Dumfries and the Nith - and I totally appreciate that - but it is al

In [5]:
dataset['train']['summary'][0]

'Clean-up operations are continuing across the Scottish Borders and Dumfries and Galloway after flooding caused by Storm Frank.'

In [6]:
ARTICLE = dataset['train']['document'][0]
summary = list(summarizer(ARTICLE, max_length=130, min_length=30, do_sample=False)[0].values())[0]
print(summary)

The number of people affected by flooding in Dumfries and Galloway and the Borders has risen to more than 1,000, the Scottish government has said.


In [7]:
from flair.data import Sentence
from flair.models import SequenceTagger
from segtok.segmenter import split_single

def NER(summary, text):
    tagger = SequenceTagger.load('ner-ontonotes')

    sentence = [Sentence(sent, use_tokenizer=True) for sent in split_single(text)]
    tagger.predict(sentence)
    text_NER = []
    for sent in sentence:
        for entity in sent.get_spans('ner'):
            text_NER.append((entity.text.strip(), entity.tag))

    sentence = [Sentence(sent, use_tokenizer=True) for sent in split_single(summary)]
    tagger.predict(sentence)
    sum_NER = []
    for sent in sentence:
        for entity in sent.get_spans('ner'):
            sum_NER.append((entity.text.strip(), entity.tag))
    return sum_NER, text_NER

In [8]:
sum_NER, text_NER = NER(summary, text)
print(sum_NER)
print("\n")
print(text_NER)

2023-09-26 10:06:47,635 SequenceTagger predicts: Dictionary with 75 tags: O, S-PERSON, B-PERSON, E-PERSON, I-PERSON, S-GPE, B-GPE, E-GPE, I-GPE, S-ORG, B-ORG, E-ORG, I-ORG, S-DATE, B-DATE, E-DATE, I-DATE, S-CARDINAL, B-CARDINAL, E-CARDINAL, I-CARDINAL, S-NORP, B-NORP, E-NORP, I-NORP, S-MONEY, B-MONEY, E-MONEY, I-MONEY, S-PERCENT, B-PERCENT, E-PERCENT, I-PERCENT, S-ORDINAL, B-ORDINAL, E-ORDINAL, I-ORDINAL, S-LOC, B-LOC, E-LOC, I-LOC, S-TIME, B-TIME, E-TIME, I-TIME, S-WORK_OF_ART, B-WORK_OF_ART, E-WORK_OF_ART, I-WORK_OF_ART, S-FAC
[('Dumfries', 'GPE'), ('Galloway', 'GPE'), ('Borders', 'LOC'), ('more than 1,000', 'CARDINAL'), ('Scottish', 'NORP')]


[('Newton Stewart', 'GPE'), ('one', 'CARDINAL'), ('Hawick', 'GPE'), ('Peeblesshire', 'GPE'), ('the Lamington Viaduct', 'FAC'), ('Newton Stewart', 'GPE'), ('the River Cree', 'LOC'), ('First', 'ORDINAL'), ('Nicola Sturgeon', 'PERSON'), ('Victoria Street', 'FAC'), ('Jeanette Tate', 'PERSON'), ('the Cinnamon Cafe', 'ORG'), ('Dumfries', 'GPE'), ('t

In [9]:
len(dataset['train'])

204045

In [11]:
summaries = []
train_summaries = []
summaries_NER = []
train_NER = []
for i in range(0,len(dataset['train'])):
    ARTICLE = dataset['train']['document'][i]
    summary = list(summarizer(ARTICLE, max_length=130, min_length=30, do_sample=False)[0].values())[0]
    summaries.append(summary)
    train_summaries.append(dataset['train']['summary'][i])
    
    sum_NER, text_NER = NER(summary, ARTICLE)
    summaries_NER.append(sum_NER)
    train_NER.append(text_NER)

2023-09-26 10:07:41,696 SequenceTagger predicts: Dictionary with 75 tags: O, S-PERSON, B-PERSON, E-PERSON, I-PERSON, S-GPE, B-GPE, E-GPE, I-GPE, S-ORG, B-ORG, E-ORG, I-ORG, S-DATE, B-DATE, E-DATE, I-DATE, S-CARDINAL, B-CARDINAL, E-CARDINAL, I-CARDINAL, S-NORP, B-NORP, E-NORP, I-NORP, S-MONEY, B-MONEY, E-MONEY, I-MONEY, S-PERCENT, B-PERCENT, E-PERCENT, I-PERCENT, S-ORDINAL, B-ORDINAL, E-ORDINAL, I-ORDINAL, S-LOC, B-LOC, E-LOC, I-LOC, S-TIME, B-TIME, E-TIME, I-TIME, S-WORK_OF_ART, B-WORK_OF_ART, E-WORK_OF_ART, I-WORK_OF_ART, S-FAC
2023-09-26 10:08:03,588 SequenceTagger predicts: Dictionary with 75 tags: O, S-PERSON, B-PERSON, E-PERSON, I-PERSON, S-GPE, B-GPE, E-GPE, I-GPE, S-ORG, B-ORG, E-ORG, I-ORG, S-DATE, B-DATE, E-DATE, I-DATE, S-CARDINAL, B-CARDINAL, E-CARDINAL, I-CARDINAL, S-NORP, B-NORP, E-NORP, I-NORP, S-MONEY, B-MONEY, E-MONEY, I-MONEY, S-PERCENT, B-PERCENT, E-PERCENT, I-PERCENT, S-ORDINAL, B-ORDINAL, E-ORDINAL, I-ORDINAL, S-LOC, B-LOC, E-LOC, I-LOC, S-TIME, B-TIME, E-TIME, I-TI

In [12]:
import pandas as pd
data = {'Original Text': train_summaries, 'NER for Original': train_NER, 'Generated Summaries': summaries, 'NER for Generated': summaries_NER}
df = pd.DataFrame(data)

In [ ]:
df.to_csv('Summaries_NER.csv', index = Falses)

In [10]:
import string
def frequencies(word, text):
    count = 0
    word = word.lower()
    text = text.lower()
    
    translator = str.maketrans('', '', string.punctuation+'\\')

    text = text.translate(translator)
    word = word.translate(translator)
    n = len(word.split())  # Get the length of the n-gram
    if n == 1:
        for i in text.split():
            if i == word:
                count += 1
    else:
        words = text.split()
        for i in range(len(words) - n + 1):
            ngram = " ".join(words[i:i + n])
            if ngram == word:
                count += 1
    return count

In [11]:
import nltk
import numpy as np
from nltk.probability import FreqDist

def returnDists(NER_list1, NER_list2, text1, text2):
    word_freq1 = []
    total_words1 = len(text1.split())
    for i in NER_list1:
        word_freq1.append((i, float(frequencies(i,text1)/total_words1)))
    word_probabilities1 = np.array(word_freq1)
    
    word_freq2 = []
    total_words2 = len(text2.split())
    for i in NER_list2:
        word_freq2.append((i, float(frequencies(i,text2)/total_words2)))
    word_probabilities2 = np.array(word_freq2)

    # Display the two probability distributions
    print("Probability Distribution for Text 1:")
    print(word_probabilities1)

    print("\nProbability Distribution for Text 2:")
    print(word_probabilities2)
    
    return word_probabilities1, word_probabilities2


In [12]:
def kl(dist1, dist2):
    return np.sum(dist1 * np.log(dist1 / dist2))

In [13]:
entities_sum = []
for i in sum_NER:
    if i[0] not in entities_sum:
        entities_sum.append(i[0])
    else:
        continue

In [14]:
entities_text = []
for i in text_NER:
    if i[0] not in entities_text:
        entities_text.append(i[0])
    else:
        continue

In [15]:
for i in range(0,len(entities_sum)):
    if i >= len(entities_sum):
        break
    if entities_sum[i] not in entities_text:
        entities_sum.remove(entities_sum[i])
        i -= 1

for i in range(0,len(entities_text)):
    if entities_text[i] not in entities_sum:
        entities_sum.append(entities_text[i])

In [16]:
print(sum_NER, entities_sum)
print(text_NER, entities_text)

[('Dumfries', 'GPE'), ('Galloway', 'GPE'), ('Borders', 'LOC'), ('more than 1,000', 'CARDINAL'), ('Scottish', 'NORP')] ['Dumfries', 'Borders', 'Scottish', 'Newton Stewart', 'one', 'Hawick', 'Peeblesshire', 'the Lamington Viaduct', 'the River Cree', 'First', 'Nicola Sturgeon', 'Victoria Street', 'Jeanette Tate', 'the Cinnamon Cafe', 'the last few days', 'Peebles', 'Scottish Borders Council', "The Labour Party's", 'Alex Rowley', 'Monday', 'first', 'Dumfries and Galloway']
[('Newton Stewart', 'GPE'), ('one', 'CARDINAL'), ('Hawick', 'GPE'), ('Peeblesshire', 'GPE'), ('the Lamington Viaduct', 'FAC'), ('Newton Stewart', 'GPE'), ('the River Cree', 'LOC'), ('First', 'ORDINAL'), ('Nicola Sturgeon', 'PERSON'), ('Victoria Street', 'FAC'), ('Jeanette Tate', 'PERSON'), ('the Cinnamon Cafe', 'ORG'), ('Dumfries', 'GPE'), ('the last few days', 'DATE'), ('Borders', 'LOC'), ('Peebles', 'ORG'), ('Scottish Borders Council', 'ORG'), ("The Labour Party's", 'ORG'), ('Scottish', 'NORP'), ('Alex Rowley', 'PERSON

In [17]:
print(text)

The full cost of damage in Newton Stewart, one of the areas worst affected, is still being assessed.
Repair work is ongoing in Hawick and many roads in Peeblesshire remain badly affected by standing water.
Trains on the west coast mainline face disruption due to damage at the Lamington Viaduct.
Many businesses and householders were affected by flooding in Newton Stewart after the River Cree overflowed into the town.
First Minister Nicola Sturgeon visited the area to inspect the damage.
The waters breached a retaining wall, flooding many commercial properties on Victoria Street - the main shopping thoroughfare.
Jeanette Tate, who owns the Cinnamon Cafe which was badly affected, said she could not fault the multi-agency response once the flood hit.
However, she said more preventative work could have been carried out to ensure the retaining wall did not fail.
"It is difficult but I do think there is so much publicity for Dumfries and the Nith - and I totally appreciate that - but it is al

In [49]:
d1, d2 = returnDists(entities_sum, entities_text, summary, text)

Probability Distribution for Text 1:
[['Dumfries' '0.04']
 ['Borders' '0.04']
 ['Scottish' '0.04']
 ['Newton Stewart' '0.0']
 ['one' '0.0']
 ['Hawick' '0.0']
 ['Peeblesshire' '0.0']
 ['the Lamington Viaduct' '0.0']
 ['the River Cree' '0.0']
 ['First' '0.0']
 ['Nicola Sturgeon' '0.0']
 ['Victoria Street' '0.0']
 ['Jeanette Tate' '0.0']
 ['the Cinnamon Cafe' '0.0']
 ['the last few days' '0.0']
 ['Peebles' '0.0']
 ['Scottish Borders Council' '0.0']
 ["The Labour Party's" '0.0']
 ['Alex Rowley' '0.0']
 ['Monday' '0.0']
 ['first' '0.0']
 ['Dumfries and Galloway' '0.04']]

Probability Distribution for Text 2:
[['Newton Stewart' '0.005']
 ['one' '0.0025']
 ['Hawick' '0.005']
 ['Peeblesshire' '0.0025']
 ['the Lamington Viaduct' '0.0025']
 ['the River Cree' '0.0025']
 ['First' '0.005']
 ['Nicola Sturgeon' '0.0025']
 ['Victoria Street' '0.0025']
 ['Jeanette Tate' '0.0025']
 ['the Cinnamon Cafe' '0.0025']
 ['Dumfries' '0.005']
 ['the last few days' '0.0025']
 ['Borders' '0.0075']
 ['Peebles' '0.0

In [50]:
dist1 = []
for i in range(0,len(d1)):
    temp = float(d1[i][1])
    dist1.append([d1[i][0], temp])

In [51]:
dist2 = []
for i in range(0,len(d2)):
    temp = float(d2[i][1])
    dist2.append([d2[i][0], temp])

In [52]:
dist1, dist2

([['Dumfries', 0.04],
  ['Borders', 0.04],
  ['Scottish', 0.04],
  ['Newton Stewart', 0.0],
  ['one', 0.0],
  ['Hawick', 0.0],
  ['Peeblesshire', 0.0],
  ['the Lamington Viaduct', 0.0],
  ['the River Cree', 0.0],
  ['First', 0.0],
  ['Nicola Sturgeon', 0.0],
  ['Victoria Street', 0.0],
  ['Jeanette Tate', 0.0],
  ['the Cinnamon Cafe', 0.0],
  ['the last few days', 0.0],
  ['Peebles', 0.0],
  ['Scottish Borders Council', 0.0],
  ["The Labour Party's", 0.0],
  ['Alex Rowley', 0.0],
  ['Monday', 0.0],
  ['first', 0.0],
  ['Dumfries and Galloway', 0.04]],
 [['Newton Stewart', 0.005],
  ['one', 0.0025],
  ['Hawick', 0.005],
  ['Peeblesshire', 0.0025],
  ['the Lamington Viaduct', 0.0025],
  ['the River Cree', 0.0025],
  ['First', 0.005],
  ['Nicola Sturgeon', 0.0025],
  ['Victoria Street', 0.0025],
  ['Jeanette Tate', 0.0025],
  ['the Cinnamon Cafe', 0.0025],
  ['Dumfries', 0.005],
  ['the last few days', 0.0025],
  ['Borders', 0.0075],
  ['Peebles', 0.0025],
  ['Scottish Borders Council', 0

In [53]:
# Sort both distributions based on the words (first element of each tuple)
dist1 = sorted(dist1, key=lambda x: x[0])
dist2 = sorted(dist2, key=lambda x: x[0])

In [54]:
for i in range(0,len(dist2)):
    if i>=len(dist2):
        break
    if dist2[i][1] == 0.0:
        dist1 = np.delete(dist1,i, axis = 0)
        dist2 = np.delete(dist2,i, axis = 0)
        i -= 1

In [55]:
dist1_keys = dist1
dist2_keys = dist2

In [56]:
for i in range(0,len(dist1)):
    dist1[i] = dist1[i][1]
for i in range(0,len(dist2)):
    dist2[i] = dist2[i][1]

In [58]:
from scipy.special import rel_entr

sum(rel_entr(dist1, dist2))

0.34421792956684494

In [50]:
d1, d2

([array(['Alex Rowley', '0.0'], dtype='<U32'),
  array(['Borders', '0.04'], dtype='<U32'),
  array(['Dumfries', '0.04'], dtype='<U32'),
  array(['Dumfries and Galloway', '0.0'], dtype='<U32'),
  array(['First', '0.0'], dtype='<U32'),
  array(['Hawick', '0.0'], dtype='<U32'),
  array(['Jeanette Tate', '0.0'], dtype='<U32'),
  array(['Monday', '0.0'], dtype='<U32'),
  array(['Newton Stewart', '0.0'], dtype='<U32'),
  array(['Nicola Sturgeon', '0.0'], dtype='<U32'),
  array(['Peebles', '0.0'], dtype='<U32'),
  array(['Peeblesshire', '0.0'], dtype='<U32'),
  array(['Scottish', '0.04'], dtype='<U32'),
  array(['Scottish Borders Council', '0.0'], dtype='<U32'),
  array(["The Labour Party's", '0.0'], dtype='<U32'),
  array(['Victoria Street', '0.0'], dtype='<U32'),
  array(['first', '0.0'], dtype='<U32'),
  array(['one', '0.0'], dtype='<U32'),
  array(['the Cinnamon Cafe', '0.0'], dtype='<U32'),
  array(['the Lamington Viaduct', '0.0'], dtype='<U32'),
  array(['the River Cree', '0.0'], dtype=

In [ ]:
"""original = []
summaries = []
ner_original = []
ner_summary = []
for i in range(0,len(dataset['train'])):
    print(i)
    original.append(dataset['train']['document'][i])
    ARTICLE = dataset['train']['document'][i]
    summary = list(summarizer(ARTICLE, max_length=130, min_length=30, do_sample=False)[0].values())[0]
    summaries.append(summary)
    print("Summaries done")
    sum_NER, text_NER = NER(summary)
    ner_original.append(text_NER)
    ner_summary.append(sum_NER)
    print("NER done")"""